# Day 3: 3.2

## XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX

## Here we are creating an DeepLearning model that can predict wether the customer is a good/bad on the basis of age and salary

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("03_Social_Network_Ads.csv")

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 5 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   User ID          400 non-null    int64 
 1   Gender           400 non-null    object
 2   Age              400 non-null    int64 
 3   EstimatedSalary  400 non-null    int64 
 4   Purchased        400 non-null    int64 
dtypes: int64(4), object(1)
memory usage: 15.8+ KB


In [4]:
data.head()

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0


In [5]:
data.isna().sum()

User ID            0
Gender             0
Age                0
EstimatedSalary    0
Purchased          0
dtype: int64

In [6]:
 df = data.iloc[:,1:]

In [7]:
df.duplicated().sum()

20

In [8]:
data.Purchased.value_counts()

0    257
1    143
Name: Purchased, dtype: int64

In [9]:
# Rules of Classification for ML:
# 1. Data must be complete
# 2. Data must be strictly numeric
# 3. Features and Label must be in the form of NumPy array
# 4. Features must be a 2d NP array
# 5. Label must be 1d NP array
# 6. Normalization of features is optional

In [10]:
features = data.iloc[:,[2,3]].values
label = data.iloc[:,4].values

In [11]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    label,
                                                    test_size=0.2, 
                                                    random_state=10)

In [12]:
# modeling
from sklearn.linear_model import LogisticRegression
log_model = LogisticRegression()
log_model.fit(X_train, y_train)

LogisticRegression()

In [13]:
# Check the quality of model
# 1. CHeck whether the model is a generalized or not
print(log_model.score(X_train,y_train))
print(log_model.score(X_test,y_test))

# 2. CHeck the quality of model with respect to CL
# SL = 0.1
# CL = 0.9

0.640625
0.65


In [14]:
# Rules for Deep Learning
# 1. Data must be complete
# 2. Data must be strictly numeric
# 3. Features and Label must be in the form of NumPy array
# 4. Features must be a 2d NP array
# 5. Label must be 2d NP array
# 6. Features must be NORMALIZED
# 7. For Binary Classification, label must be represented as 0 / 1 (pd.replace)
# 8. For Multi-class classification, label must be  DISCRETE NUMERICAL

In [15]:
features = data.iloc[:,[2,3]].values
label = data.iloc[:,[4]].values

In [16]:
from sklearn.preprocessing import RobustScaler
rbFeatures=RobustScaler()
features = rbFeatures.fit_transform(features)

In [17]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(features, 
                                                    label, 
                                                    test_size=0.2,
                                                    random_state=10)

In [18]:
# Modeling
# Step1 : Architect the model
# step 1.1: Create a Sequential Model

seq_model = tf.keras.Sequential()

# Step 1.2: Create Dense Layer
# units = No. of neurons in Hidden Layer
# activation = Which Activation Function to Apply
# input_shape = Number of colums in FeatureArray


seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1), input_shape = (2,)))
seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
seq_model.add(tf.keras.layers.Dense(units=6, activation=tf.keras.layers.LeakyReLU(alpha=0.1)))
seq_model.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [19]:
# Compiling
# Step2: Compile the mode:

seq_model.compile(optimizer="sgd", loss='binary_crossentropy', metrics=["accuracy"])

In [20]:
#seq_model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test))

In [21]:
class MyThresholdCallBack(tf.keras.callbacks.Callback):
    def __init__(self,cl):
        super(MyThresholdCallBack, self).__init__()
        self.cl = cl
    
    def on_epoch_end(self, epoch, logs=None):
        test_score = logs["val_accuracy"]
        train_score = logs["accuracy"]
        
        if test_score > train_score and test_score > self.cl:
            self.model.stop_training = True

In [22]:
accuracy_Threshold = MyThresholdCallBack(cl=0.92)

seq_model.fit(X_train, y_train, epochs=1000, validation_data=(X_test, y_test), callbacks=[accuracy_Threshold])

Epoch 1/1000
10/10 [==============================] - 1s 33ms/step - loss: 0.7087 - accuracy: 0.3125 - val_loss: 0.7022 - val_accuracy: 0.3125
Epoch 2/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.6998 - accuracy: 0.4594 - val_loss: 0.6947 - val_accuracy: 0.5375
Epoch 3/1000
10/10 [==============================] - 0s 12ms/step - loss: 0.6930 - accuracy: 0.5688 - val_loss: 0.6892 - val_accuracy: 0.5750
Epoch 4/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.6876 - accuracy: 0.6250 - val_loss: 0.6845 - val_accuracy: 0.6875
Epoch 5/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.6831 - accuracy: 0.6594 - val_loss: 0.6804 - val_accuracy: 0.7000
Epoch 6/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.6793 - accuracy: 0.6812 - val_loss: 0.6769 - val_accuracy: 0.7125
Epoch 7/1000
10/10 [==============================] - 0s 8ms/step - loss: 0.6759 - accuracy: 0.7188 - val_loss: 0.6738 - val_accuracy: 0.7375
Epoc

10/10 [==============================] - 0s 7ms/step - loss: 0.5495 - accuracy: 0.7875 - val_loss: 0.5411 - val_accuracy: 0.7875
Epoch 59/1000
10/10 [==============================] - 0s 12ms/step - loss: 0.5455 - accuracy: 0.7844 - val_loss: 0.5368 - val_accuracy: 0.7875
Epoch 60/1000
10/10 [==============================] - 0s 12ms/step - loss: 0.5414 - accuracy: 0.7937 - val_loss: 0.5324 - val_accuracy: 0.7750
Epoch 61/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.5372 - accuracy: 0.7969 - val_loss: 0.5279 - val_accuracy: 0.7875
Epoch 62/1000
10/10 [==============================] - 0s 6ms/step - loss: 0.5329 - accuracy: 0.8062 - val_loss: 0.5232 - val_accuracy: 0.8000
Epoch 63/1000
10/10 [==============================] - 0s 9ms/step - loss: 0.5285 - accuracy: 0.8188 - val_loss: 0.5185 - val_accuracy: 0.8000
Epoch 64/1000
10/10 [==============================] - 0s 10ms/step - loss: 0.5240 - accuracy: 0.8250 - val_loss: 0.5136 - val_accuracy: 0.8125
Epoch 65/

In [23]:
seq_model.evaluate(X_train,y_train)

10/10 [==============================] - 0s 3ms/step - loss: 0.4507 - accuracy: 0.8875


[0.45069199800491333, 0.887499988079071]

In [24]:
seq_model.evaluate(X_test,y_test)

3/3 [==============================] - 0s 3ms/step - loss: 0.4381 - accuracy: 0.9250


[0.4381323754787445, 0.925000011920929]

In [25]:
(seq_model.predict(features) > 0.5).astype('int32')

13/13 [==============================] - 0s 2ms/step


array([[0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [1],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
       [0],
    

In [26]:
from sklearn.metrics import confusion_matrix
confusion_matrix(label, (seq_model.predict(features) > 0.5).astype('int32'))

13/13 [==============================] - 0s 2ms/step


array([[243,  14],
       [ 28, 115]], dtype=int64)

In [27]:
from sklearn.metrics import classification_report
print(classification_report(label, (seq_model.predict(features) > 0.5).astype('int32')))

13/13 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.90      0.95      0.92       257
           1       0.89      0.80      0.85       143

    accuracy                           0.90       400
   macro avg       0.89      0.87      0.88       400
weighted avg       0.89      0.90      0.89       400



In [28]:
# Deployment with Input Example:

# 1. Getting user input in the form of float
ageInYears = float(input("Enter age of Customer: "))
salaryCust = float(input("Enter salary of Customer: "))

# 2. Converting input to 2d array since the same format was used for training this model
custArray = np.array([[ageInYears, salaryCust]])

# 3. Apply Standardization scince the same was done during the training of model
scaledCustData = rbFeatures.transform(custArray)

# 4. Predicting the salary
#purchase = seq_model.predict(scaledCustData)

# 5. Inverse Transform Label
actualPurch = ((seq_model.predict(scaledCustData) > 0.5).astype('int32')).item()

def final_rep(x):
    if actualPurch == 0:
        return "No"
    if actualPurch == 1:
        return "Yes"

print(f"The Customer wih {ageInYears} and salary {salaryCust} is going to purchare?: {final_rep(actualPurch)}")

Enter age of Customer: 19
Enter salary of Customer: 76000
1/1 [==============================] - 0s 30ms/step
The Customer wih 19.0 and salary 76000.0 is going to purchare?: No


In [29]:
scaledCustData

array([[-1.10769231,  0.13333333]])